In [64]:
import chromadb
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import SummaryIndex

from llama_index.llms.openai import OpenAI
import os 
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from llama_index.core.objects import ObjectIndex


chroma_client = chromadb.PersistentClient(path="chroma_db")
chroma_collection = chroma_client.get_or_create_collection("quickstart_no_metadata")


In [65]:
function_llm = OpenAI(model="gpt-4",  api_base=os.environ.get("CLOUDFLARE_API_BASE_URL", None))
Settings.llm = OpenAI(model="gpt-4", api_base=os.environ.get("CLOUDFLARE_API_BASE_URL", None))
Settings.embed_model = OpenAIEmbedding(api_base=os.environ.get("CLOUDFLARE_API_BASE_URL", None))

In [66]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
)



In [74]:
engine = index.as_query_engine(llm=Settings.llm)

response = engine.query("summarize the document for the RFP related toconstruction?")
print(response)

The document is a Request for Proposal (RFP) for a construction project managed by the Incourage Community Foundation. It outlines the requirements for the proposal, including details about subcontracted work, ability to meet timelines, tax-exempt status, and fees for both pre-construction and construction phases. The document also specifies that the proposer must manage material purchases to take advantage of the owner's tax-exempt status. 

The proposal must include information about the firm, its organization, personnel, hiring practices, qualifications, experience, and any self-performed work. The document also requests examples of the firm's commitment to diversity, sustainability, and connection to place. 

Proposals were to be submitted by 4:00 p.m. on Friday, February 13, 2015, either by email or hard copy. There were also opportunities for prospective proposers to participate in a site visit and a Q&A webinar. 

The final selection would be based on the qualifications and expe

In [68]:
response.metadata


{'6fd2f931-b87f-4844-87bb-d52831ba290c': {'page_label': '6',
  'file_name': 'kitchenai/5faf76ea-ed89-45ae-b977-16ba61427b25/construction-rfp.pdf',
  'file_id': 33,
  'source': 'kitchenai_cookbook',
  'file_label': 'construction'},
 '4a88ab8a-890f-4f91-979b-f2b661f88abe': {'page_label': '4',
  'file_name': 'kitchenai/fa102c5e-2e00-429c-922f-b7976d584b0d/consultancy-rfp.pdf',
  'file_id': 31,
  'source': 'kitchenai_cookbook',
  'file_label': 'consultancy'}}

In [69]:

file_label = "construction"

label_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    filter={"file_label": file_label}
)


In [70]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=engine,
        metadata=ToolMetadata(
            name="rfp_collection",
            description=(
                "A catch all tool for any question."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=label_index.as_query_engine(llm=Settings.llm),
        metadata=ToolMetadata(
            name=file_label,
            description=(
                f"Provides information about the current working RFP: {file_label}"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]




In [71]:
local_agent = OpenAIAgent.from_tools(
    query_engine_tools,
    llm=function_llm,
    verbose=True,
    system_prompt=f"""\
You are an expert in analyzing RFPs and breaking down their requirements from previous incoming RFP requests.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
)


obj_index = ObjectIndex.from_objects(
    query_engine_tools,
    index_cls=VectorStoreIndex,
)

top_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    system_prompt=""" \
You are an RFP expert designed to help the proposal team understand the requirements of incoming RFPs.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\
""",
    verbose=True,
)



In [72]:
response = top_agent.query(f"build a report on the requirements of the incoming RFP: construction, include a list of requirements and a comparison to the advertising and consultancy rfps")


Added user message to memory: build a report on the requirements of the incoming RFP: construction, include a list of requirements and a comparison to the advertising and consultancy rfps
=== Calling Function ===
Calling function: construction with args: {
"input": "What are the requirements of the incoming RFP: construction?"
}
Got output: The incoming Request for Proposal (RFP) for construction requires the proposers to demonstrate prior experience with Integrated Project / Construction Management-at-risk delivery methods, with at least one project over $8M in construction costs. They should also have prior experience with delivering projects that have achieved sustainability benchmarks such as LEED, Living Building Challenge. The proposers should have qualified personnel with experience collaborating with Architects and Engineers during Design Phases and qualified Cost Estimators on staff. The successful proposer should be available to begin work immediately upon selection and be pr

In [73]:
response.response

"The requirements for the incoming RFP for construction, advertising, and consultancy are as follows:\n\n**Construction RFP Requirements:**\n1. Demonstrated prior experience with Integrated Project / Construction Management-at-risk delivery methods, with at least one project over $8M in construction costs.\n2. Prior experience with delivering projects that have achieved sustainability benchmarks such as LEED, Living Building Challenge.\n3. Qualified personnel with experience collaborating with Architects and Engineers during Design Phases and qualified Cost Estimators on staff.\n4. Availability to begin work immediately upon selection and preparedness for project activities to commence.\n5. Ability to work within the project timeline, with the understanding that there may be adjustments as the project evolves.\n6. Non-selection as the Construction Manager under this RFP does not impact the proposer’s opportunity to participate in the Project as a sub-contractor.\n\n**Advertising RFP Re